# Filter annotation fragments based on motif score (input 20x)

In [1]:
source ../config_duke.sh -v

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect



**Check**

In [3]:
ls ${FD_RES}/annotation_fragment/Input1_20x

chr10.bed.gz  chr15.bed.gz  chr1.bed.gz   chr3.bed.gz  chr8.bed.gz
chr11.bed.gz  chr16.bed.gz  chr20.bed.gz  chr4.bed.gz  chr9.bed.gz
chr12.bed.gz  chr17.bed.gz  chr21.bed.gz  chr5.bed.gz  chrX.bed.gz
chr13.bed.gz  chr18.bed.gz  chr22.bed.gz  chr6.bed.gz  chrY.bed.gz
chr14.bed.gz  chr19.bed.gz  chr2.bed.gz   chr7.bed.gz  target_PER1.bed.gz


In [2]:
ls ${FD_RES}/annotation_fragment/Input1_20x/chr17.bed.gz

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/Input1_20x/chr17.bed.gz


In [4]:
zcat ${FD_RES}/annotation_fragment/Input1_20x/chr17.bed.gz | head -5 | expand -t 12

chr17       107410      108464      1           chr17       107391      107414      TBX/4       8.2847      4
chr17       107410      108464      1           chr17       107408      107426      ZNF250      2.3414      16
chr17       107410      108464      1           chr17       107416      107435      ZNF143      8.6671      19
chr17       107410      108464      1           chr17       107422      107452      GC-tract    11.2233     30
chr17       107410      108464      1           chr17       107422      107432      GLI         7.9971      10

gzip: stdout: Broken pipe


In [5]:
zcat ${FD_RES}/annotation_fragment/Input1_20x/chr17.bed.gz |\
    head -5 |\
    awk '$9>8' |\
    expand -t 12


gzip: stdout: Broken pipe
chr17       107410      108464      1           chr17       107391      107414      TBX/4       8.2847      4
chr17       107410      108464      1           chr17       107416      107435      ZNF143      8.6671      19
chr17       107410      108464      1           chr17       107422      107452      GC-tract    11.2233     30


In [14]:
source ../config_duke.sh

sbatch -p ${NODE} \
    --array 0-23 \
    --mem 8G \
    -o ${FD_LOG}/filter_annotate_fragment_chrom_input20x.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
CHROMS=($(seq 1 22) X Y)
CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
TARGET=${CHROM}

### set input and output
FD_BEDS=($(ls -d ${FD_RES}/annotation_fragment/Input?_20x/))
FN_BED=${TARGET}.bed.gz

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### loop through each sample and count each fragment of a chromosome
echo "RUN: loop through samples; annotate fragment"

for FD_BED in ${FD_BEDS[@]}; do
    SAMPLE=$(basename ${FD_BED})
    FD_OUT=${FD_RES}/annotation_fragment/filter_motif_score095/${SAMPLE}
    FN_OUT=${TARGET}.bed.gz
    
    ### START Message
    echo ++++++++++++++++++++++++++++++++++++++++++++++++
    echo "Input  file: " ${FD_BED}/${FN_BED}
    echo "Output file: " ${FD_OUT}/${FN_OUT}
    echo
    echo "Show the first few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    zcat ${FD_BED}/${FN_BED} | head -n 3
    echo
    
    ### init: create output folder if not exist
    mkdir -p ${FD_OUT}
    
    ### filteration
    #zcat ${FD_BED}/${FN_BED} | head -10 | awk '$9>8' | gzip > ${FD_OUT}/${FN_OUT}
    zcat ${FD_BED}/${FN_BED} | awk '$9>10.81' | gzip > ${FD_OUT}/${FN_OUT}
    
    ### END Message
    echo "Show the first few lines of the output file"
    echo ${FD_OUT}/${FN_OUT}
    zcat ${FD_OUT}/${FN_OUT} | head -3
    echo
done

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 26237332


In [15]:
cat ${FD_LOG}/filter_annotate_fragment_chrom_input20x.0.txt

Slurm Array Index:  0
Time Stamp:         10-02-21+14:56:02

RUN: loop through samples; annotate fragment
++++++++++++++++++++++++++++++++++++++++++++++++
Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/Input1_20x//chr1.bed.gz
Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input1_20x/chr1.bed.gz

Show the first few lines of the input file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/Input1_20x//chr1.bed.gz
chr1	13802	14757	1	chr1	13790	13810	ZNF335	6.8996	8
chr1	13802	14757	1	chr1	13792	13806	NR/3	8.1207	4
chr1	13802	14757	1	chr1	13794	13814	GC-tract	6.2461	12

Show the first few lines of the output file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input1_20x/chr1.bed.gz
chr1	13802	14757	1	chr1	13937	13949	INSM1	11.5497	12
chr1	13802	14757	1	chr1	13971	13980	AP1/1	10.9638	9
chr1	13802	14757	1	chr1	14035	14050	NFI/1	13.4934	15

In [16]:
ls ${FD_RES}/annotation_fragment

filter_motif_score095  Input3      target_PER1_MA0099.3.bed  TFX4_Dex
Input                  Input3_20x  target_PER1_MA0113.3.bed  TFX4_DMSO
Input1                 Input4      TFX2_Dex                  TFX5_Dex
Input1_20x             Input4_20x  TFX2_DMSO                 TFX5_DMSO
Input2                 Input5      TFX3_Dex                  TFX_Dex
Input2_20x             Input5_20x  TFX3_DMSO                 TFX_DMSO


In [17]:
ls ${FD_RES}/annotation_fragment/filter_motif_score095

Input1      Input2_20x  Input4      Input5_20x  TFX3_Dex   TFX4_DMSO
Input1_20x  Input3      Input4_20x  TFX2_Dex    TFX3_DMSO  TFX5_Dex
Input2      Input3_20x  Input5      TFX2_DMSO   TFX4_Dex   TFX5_DMSO
